In [306]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import datetime
from graphviz import Digraph
import time
from tqdm import tqdm

In [307]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [308]:
df = pd.read_csv('/Users/kost/Desktop/решено/data/statistics-07-20.csv',sep=';')
train_ops = pd.read_csv('/Users/kost/Desktop/решено/data/poezd.csv', sep=';', encoding='utf-8')
car_ops = pd.read_csv('/Users/kost/Desktop/решено/data/codes.csv',sep=';', encoding='utf-8')

In [309]:
ndf = df.sort_values(by=['operation_st_id','car_number','operation_date']).drop_duplicates().reset_index(drop=True)

### Первичный анализ датасета

In [310]:
operation_code = 2
print(train_ops[train_ops.index_code == operation_code].to_string(index=False))
print(car_ops[car_ops.index_code == operation_code].to_string(index=False))

 index_code mnem_code                                                         description
          2        От Отпр. поезда со станции (без сдачи на дорогу,отделение,участок ДНЦ)
 index_code mnem_code                            description
          2      ОТПР ОТПРАВЛЕНИЕ ВАГОНА СО СТАНЦИИ         


In [311]:
# уникальные станции по операциям
df.operation_st_id.value_counts()

2,000,038,976.00    207099
2,000,038,600.00    146427
2,000,037,862.00    116936
2,001,933,494.00     96307
2,001,930,816.00     82794
                     ...  
2,000,037,640.00         1
2,000,036,452.00         1
2,000,036,458.00         1
2,000,035,312.00         1
2,000,036,334.00         1
Name: operation_st_id, Length: 744, dtype: int64

In [312]:
# уникальные вагоны по операциям
df.car_number.value_counts()

55864821    310
55822928    290
55927537    287
55626428    273
55701130    272
           ... 
60175320      1
95378899      1
29064912      1
57448243      1
60025954      1
Name: car_number, Length: 441248, dtype: int64

### Избавляемся от пропусков в operation_car

In [313]:
for i in ndf.operation_train[ndf.operation_car.isna()].value_counts().index:
    print(i,train_ops.description[train_ops.index_code == i].values)

2.0 ['Отпр. поезда со станции (без сдачи на дорогу,отделение,участок ДНЦ)']
62.0 ['Отпр. поезда со станции со сдачей на соседний участок']
22.0 ['Отпр. поезда co станции со сдачей на другую дорогу']
42.0 ['Отпр. поезда со станции со сдачей на соседнее отделение']


In [314]:
# все пропуски можно заменить операцией отправления
ndf.operation_car = ndf.operation_car.fillna(2)

### Избавляемся от пропусков в danger

In [315]:
ndf.danger = ndf.danger.fillna(0)

### Отрисовка графов последовательностей операций

In [316]:
def ops_tr_for_set_of_carts(cartsSet: set) -> pd.DataFrame:
    """
    Builds DF with historical data of changes in states from one operation to another for a set of carts IDs.
    :param cartsSet: set with carts numbers from nom_vag_op
    :return: dataframe with following format (note that state changes are not unique)
        from 	to
    0 	3 	    4
    1 	4 	    3
    2 	3 	    4
    3 	4 	    3
    ...
    """
    stateChangeDF = pd.DataFrame(columns=['from', 'to'])
    iter = 0
    for cartID in tqdm(cartsSet):
        iter += 1
        opsListCart = list(ndf.operation_car[ndf.car_number == cartID])
        state_change = pd.DataFrame({'from': opsListCart[:-1], 'to': opsListCart[1:]})
        stateChangeDF = pd.concat(
            [stateChangeDF, state_change],
            axis=0,
            join="outer",
            ignore_index=True,
            keys=None,
            levels=None,
            names=None,
            verify_integrity=False,
            copy=True,
        )
    return stateChangeDF


def count_ops_transitions(cartsSet: set) -> dict:
    """
    Builds a dict with unique operation changes and state change frequency.
    :param cartsSet: set with carts numbers from nom_vag_op
    :return: dict with unique operations and their quantity. See example below:
    {'3->4': 2, '4->3': 2, '3->2': 1, '2->80': 1}
    """
    sChangeList = []
    linksDict: dict = {}
    sChangeDF = ops_tr_for_set_of_carts(cartsSet)
    for i in range(sChangeDF.shape[0]):
        src = str(list(sChangeDF.loc[i])[0])
        trg = str(list(sChangeDF.loc[i])[1])
        sChangeList.append(src + "->" + trg)
    for entry in sChangeList:
        key = str(entry)
        if key in linksDict.keys():
            linksDict[key] += 1
        else:
            linksDict[key] = 1
    return linksDict

In [317]:
# cars
# carSet = {37843901}
# carSet = {94033792}

# stations
# option A - 
# carSet = set(ndf.car_number[ndf.operation_st_id == 2000038976])

# option B - dropp all ops chains with less then 3
st = ndf.car_number[ndf.operation_st_id == 2000038976].value_counts()
carSet = set(st[st > 2].index)

In [318]:
sdict = count_ops_transitions(carSet)

DG = Digraph('finite_state_machine', filename='ops_fsm_rzd')
DG.attr(rankdir='LR', size='8,5')

for key in sdict:
    src = car_ops[car_ops.index_code == float(key.split('->')[0])].values[0][1]
    dst = car_ops[car_ops.index_code == float(key.split('->')[1])].values[0][1]
    DG.edge(src, dst, weight=str(sdict[key]), label=str(sdict[key]))

DG.view()

100%|██████████| 38496/38496 [04:11<00:00, 153.00it/s]


'ops_fsm_rzd.pdf'

### Профайлер

In [ ]:
## profiler code
dt = datetime.datetime.now().strftime("%m/%d/%y - %H:%M")
profile = ProfileReport(ndf, title=dt, explorative=True)
fname = 'pd-report.html'
profile.to_file(output_file=fname)

### =================

In [319]:
ndf

,index_train,length,car_number,destination_esr,adm,danger,gruz,loaded,operation_car,operation_date,operation_st_esr,operation_st_id,operation_train,receiver,rodvag,rod_train,sender,ssp_station_esr,ssp_station_id,tare_weight,weight_brutto
0,"880,106,902,880,202.00",NaN,30891014,"880,303.00",NaN,0.00,NaN,2.00,3.00,2020-07-22 13:00:00,"880,303.00","2,000,035,070.00",4.00,NaN,90.00,83.00,NaN,"880,202.00",NaN,NaN,47.00
1,"880,106,902,880,202.00",NaN,30893911,"880,303.00",NaN,0.00,NaN,2.00,3.00,2020-07-22 13:00:00,"880,303.00","2,000,035,070.00",4.00,NaN,90.00,83.00,NaN,"880,202.00",NaN,NaN,47.00
2,NaN,1.38,29068855,"893,500.00",20.00,0.00,"421,034.00",NaN,18.00,2020-07-22 13:37:00,"880,407.00","2,000,035,090.00",NaN,"91,860,990.00",20.00,NaN,"33,977,721.00",NaN,NaN,269.00,NaN
3,"880,407,949,880,106.00",1.38,29068855,"893,500.00",NaN,0.00,"521,016.00",2.00,4.00,2020-07-22 22:30:00,"880,407.00","2,000,035,090.00",5.00,"91,860,990.00",20.00,72.00,"33,977,721.00","880,106.00","2,000,035,130.00",NaN,240.00
4,"880,407,949,880,106.00",1.38,29068855,"893,500.00",NaN,0.00,"521,016.00",NaN,2.00,2020-07-23 00:03:00,"880,407.00","2,000,035,090.00",2.00,"91,860,990.00",20.00,72.00,"33,977,721.00","880,106.00","2,000,035,130.00",NaN,240.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4188133,NaN,NaN,64437627,"917,207.00",NaN,0.00,NaN,2.00,79.00,2020-07-26 22:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4188134,"969,004,044,968,209.00",1.00,64437627,"917,207.00",NaN,0.00,"161,113.00",2.00,4.00,2020-07-26 23:43:00,NaN,NaN,5.00,"161,246.00",60.00,NaN,"52,682,351.00","968,209.00","2,000,038,610.00",NaN,NaN
4188135,"969,004,044,968,209.00",1.00,64437627,"917,207.00",NaN,0.00,"161,113.00",NaN,2.00,2020-07-27 00:36:00,NaN,NaN,2.00,"161,246.00",60.00,52.00,"52,682,351.00","968,209.00","2,000,038,610.00",NaN,"1,036.00"
4188136,NaN,1.00,65635427,"693,903.00",20.00,0.00,"351,043.00",NaN,13.00,2020-07-30 16:30:00,NaN,NaN,NaN,0.00,60.00,NaN,"1,373,772.00",NaN,NaN,244.00,NaN
